In [1]:
#The estimation of the MLE is based on the material from Quantecon

In [2]:
#First, we need the following imports

In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp
from scipy.special import factorial
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from statsmodels.api import Poisson
from statsmodels.api import Probit
from statsmodels.api import Logit
from scipy import stats
from scipy.stats import norm
from statsmodels.iolib.summary2 import summary_col

In [23]:
%%html
<p>
Consider the probit model where the dependent variable ($Y$) is binary, and the independent variables are given by the vector $X$. 
In addition, assume that $Pr(Y=1|X)= Φ(X^T β)$ where $Φ$ is the CDF of the standard normal distribution. 
</p>

<p>
The objective is to maximize the likelihood function over the parameters to find the Maximum Likelihood Estimator (MLE). 
To find the MLE estimator we maximize the next Log-Likelihood function:
</p>

<p>
$$
\ln L(\beta \mid X, Y)=\sum_{i=1}^n\left[y_i \ln \Phi\left(x_i^{\prime} \beta\right)+\left(1-y_i\right) \ln \left(1-\Phi\left(x_i^{\prime} \beta\right)\right)\right]
$$
</p>


In [24]:
#Define the Profit Log-Likelihood Function from https://nbviewer.org/github/VHRanger/MLE-tutorial/blob/master/Implementing%20and%20vectorizing%20a%20Maximum%20Likelihood%20model%20with%20scipy--1.ipynb

def LogLikeProbit(betas, y, x):
    """
    Probit Log Likelihood function
    Very slow naive Python version
    Input:
        betas is a np.array of parameters
        y is a one dimensional np.array of endogenous data
        x is a 2 dimensional np.array of exogenous data
            First vertical column of X is assumed to be constant term,
            corresponding to betas[0]
    returns:
        negative of log likehood value (scalar)
    """
    result = 0
    #Sum operation
    for i in range(0, len(y)):
        #Get X'_i * Beta value
        xb = np.dot(x[i], betas)
        
        #compute both binary probabilities from xb     
        #Add to total log likelihood
        llf = y[i]*np.log(norm.cdf(xb)) + (1-y[i])*np.log(1 - norm.cdf(xb))
        result += llf
    return -result

In [25]:
#####################
#ARTIFICIAL DATA
######################

#sample size
n = 1000

#random generators
z1 = np.random.randn(n)
z2 = np.random.randn(n)

#create artificial exogenous variables 
x1 = 0.8*z1 + 0.2*z2
x2 = 0.2*z1 + 0.8*z2
#create error term
u = 2*np.random.randn(n)

#create endogenous variable from x1, x2 and u
ystar = 0.5 + 0.75*x1 - 0.75*x2 + u

#create latent binary variable from ystar
def create_dummy(data, cutoff):
    result = np.zeros(len(data))
    for i in range(0, len(data)):
        if data[i] >= cutoff:
            result[i] = 1
        else:
            result[i] = 0
    return result

#get latent LHS variable
y = create_dummy(ystar, 0.5)

#prepend vector of ones to RHS variables matrix
#for constant term
const = np.ones(n)
x = np.column_stack((const, np.column_stack((x1, x2))))

In [26]:
from scipy.optimize import minimize


In [28]:
#create beta hat vector to maximize on
#will store the values of maximum likelihood beta parameters
#Arbitrarily initialized to all zeros
bhat = np.zeros(len(x[0]))

#unvectorized MLE estimation
probit_est = minimize(LogLikeProbit, bhat, args=(y,x), method='nelder-mead')

#print vector of maximized betahats
probit_est['x']

array([ 0.02607502,  0.41013366, -0.36970816])

In [29]:
import statsmodels.tools.numdiff as smt
import scipy as sc

#Get inverse hessian for Cramer Rao lower bound
b_estimates = probit_est['x']
Hessian = smt.approx_hess3(b_estimates, LogLikeProbit, args=(y,x))
invHessian = np.linalg.inv(Hessian)

#Standard Errors from C-R LB
#from diagonal elements of invHessian
SE = np.zeros(len(invHessian))
for i in range(0, len(invHessian)):
    SE[i] =  np.sqrt(invHessian[i,i])
    
#t and p values
t_statistics = (b_estimates/SE)
pval = (sc.stats.t.sf(np.abs(t_statistics), 999)*2)

print("Beta Hats: ", b_estimates)
print("SE: ", SE)
print("t stat: ", t_statistics)
print("P value: ", pval)

Beta Hats:  [ 0.02607502  0.41013366 -0.36970816]
SE:  [0.04046088 0.05752986 0.0550724 ]
t stat:  [ 0.64445017  7.12905693 -6.71313008]
P value:  [5.19431501e-01 1.93558832e-12 3.18915601e-11]


In [36]:
import statsmodels 
#Using other routine
stats_probit = Probit(y, x).fit()
print(stats_probit.summary())


Optimization terminated successfully.
         Current function value: 0.659063
         Iterations 4
                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                         Probit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Tue, 18 Oct 2022   Pseudo R-squ.:                 0.04884
Time:                        23:09:23   Log-Likelihood:                -659.06
converged:                       True   LL-Null:                       -692.91
Covariance Type:            nonrobust   LLR p-value:                 2.007e-15
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0261      0.040      0.644      0.519      -0.053       0.105
x1             0.4102      0.